# Word2Vec

* [gensim](https://radimrehurek.com/gensim/index.html)を使った実装
* 公式のドキュメント  
https://radimrehurek.com/gensim/models/word2vec.html
* 学習済みモデルに対して追加学習もできる

<a href="https://colab.research.google.com/github/fuyu-quant/Data_Science/blob/main/Natural_Language_processing/Text_Embedding(japanese)/Word2Vec(japanese).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
# gensimのinstall(colabでは初めからinstallされているため不要)
#!pip install --upgrade gensim
# gensimを使うためにnumpyをアップグレード(再起動をする必要がある)
!pip install --upgrade numpy
# 日本語の分かち書きを得るためにMeCabもinstall
!pip install SudachiPy==0.5.4 ginza==4.0.6 ja-ginza==4.0.0

In [64]:
# colabで動かすために必要なコード
import pkg_resources, imp
imp.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py'>

In [78]:
# Word2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import spacy

## データの用意
* 大量の文章で学習させた方が精度が良くなる

In [67]:
text = "２ 【事業等のリスク】 当社及び当社グループの事業その他に関するリスクについて、投資者の判断に重要な影響を及ぼす可能性があると 考えられる主な事項や、その他リスク要因に該当しな事項であっても、投資者の投資判断上、重要であると考えら れる事項について記載しております。また、これらのリスクは互いに独立するものではなく、ある事象の発生により 他の様々なリスクが増大する可能性があることについてもご留意ください。なお、当社は、これらリスクの発生可能 性を認識したうえで、発生を回避するための施策を講じるとともに、発生した場合には迅速かつ適切な対応に努める 所存であります。 本項においては、将来に関する事項が含まれておりますが、当該事項は有価証券報告書提出日現在において判断し たものであります。 (1) 経営環境等に関するリスク 当社グループを取り巻く経営環境が大きく変動した場合、当社グループの経営成績及び財政状態に影響を及ぼす 可能性があります。具体的には以下の通りであります。 ① 近時の国内外の経済金融環境 当社グループは、国際金融市場の変動や国内外の景気の下振れ、資源価格の急激な下落等の国内外の金融経済 環境の変動に対して、リスク管理体制の整備・高度化も含めた様々な対応策を講じております。しかしながら、 当社グループの想定を上回る変動が生じた場合には、後記「(2)当社グループの業務に内包されるリスク」に記載 の信用リスク、市場リスク及び流動性リスク等が顕在化し、当社グループの経営成績及び財政状態に影響を及ぼ す可能性があります。 ② ロシア・ウクライナ情勢の深刻化・長期化に関するリスク 2022年２月に発生したロシアによるウクライナへの侵攻に対し、日本、米国、欧州などの世界各地で対ロシア 制裁措置が講じられたこと、また、ロシアによる西側諸国への対抗措置が講じられたこと等で、ロシア・ウクラ イナ両国における市民生活や経済活動に甚大な影響が生じています。当社グループでは、こうした状況を踏ま え、当連結会計年度においてロシア関連与信に対する貸倒引当金を計上しております。また、当社の持分法適用 会社であるSMBC Aviation Capital Limitedにおいて、ロシア向け航空機リース資産の減損を実施しております。 ロシア・ウクライナ情勢については、現時点で収束が見込み難く、その影響が深刻化・長期化した場合、追加 的な信用コストや減損が発生する等の可能性があります。また、資源価格の高騰やサプライチェーンを通じた世 界経済への悪影響が想定以上に大きくなる場合や金融市場が混乱する場合には、更なる信用コストの発生や当社 グループが保有する金融商品の評価損などが発生する可能性があります。 ③ 災害等の発生に関するリスク 当社グループは、国内外の店舗、事務所、電算センター等の施設において業務を行っておりますが、これらの 施設は、地震等の自然災害、停電、テロ等による被害を受ける可能性があります。また、各種感染症の流行によ り、当社グループの業務運営に支障が生じる可能性があります。当社グループは、不測の事態に備えたコンティ ンジェンシープランを策定しておりますが、被害の程度によっては、当社グループの業務が停止し、当社グルー プの業務運営や経営成績及び財政状態に影響を及ぼす可能性があります。 ― 17 ― ④ 新型コロナウイルス感染症の感染拡大に関するリスク 2019年12月以降、世界各国で新型コロナウイルス感染症の流行が拡大し、これに対し、日本を含む世界各国 で、外出や渡航の禁止・制限、商業施設や生産工場の営業自粛・休業など、様々な感染拡大の防止措置がとられ たことで、市民生活や経済活動が著しく制限され、企業業績の悪化や個人消費の低下など、日本経済、世界経済 に深刻な影響が生じました。 新型コロナウイルスのワクチンの開発・普及に伴い、日本を含む世界各国で、市民生活や経済活動の正常化に 向けた動きが進んでいますが、新たな変異株の流行等により感染収束が遅れ、国内外の経済の不安定な状態が長 引く、もしくは更に悪化する可能性や金融市場の混乱が再び起こる可能性があります。 新型コロナウイルス感染症の感染拡大による影響により、国内外の経済の不安定な状態が長引く、もしくは更 に悪化した場合、取引先の業況悪化等により与信関係費用及び不良債権残高が増加する可能性があるほか、お客 さまの資金繰り支援として新規・追加融資の要請に応えることで、リスクアセットが増加し自己資本比率が低下 する可能性があります。また、金融市場の混乱が再び起きた場合、当社グループの資金繰り状況が悪化するほ か、株式等、当社グループが保有する金融商品に減損又は評価損が発生する可能性があります。加えて、経済活 動の抑制によるビジネス機会の喪失等により、当社グループの戦略遂行に支障が生じる可能性があります。 当社グループは、融資・決済サービスなど社会インフラとして金融機関に求められるサービスを継続的に提供 するという責任を果たすべく、新型コロナウイルス感染症の感染拡大の中でも業務を継続しており、そのために 十分な感染防止策を講じております。しかしながら、同一拠点にて多数の従業員が同時に罹患した場合には、一 時的に業務継続に支障が生じるなど、当社グループが十分な金融サービスを提供できなくなる可能性がありま す。 ⑤ サステナビリティを意識した動きの加速に関するリスク 現在、世界は様々な社会課題に直面しています。特に気候変動や人権問題などは世界規模で深刻化しており、 その影響は広範に及び、かつ不確実性を伴います。こうしたなか、ＳＤＧｓ(※)の達成やＥＳＧへの関心が高ま るなど、持続可能な社会の実現に向けて、政官民を挙げた取組みが求められています。 当社グループは、後記「第４ 提出会社の状況 ４ コーポレート・ガバナンスの状況等 （１） コーポレ ート・ガバナンスの概要 ③ 内部統制システム ホ．サステナビリティへの取組み」に記載のとおり、サステ ナビリティの実現に向けた取組みを行っております。しかしながら、これらの取組みが奏功しない、もしくは不 十分である場合、社会的な批判の高まりを受けたお客さま及び市場等からの信用失墜等により、当社グループの 経営成績及び財政状態に影響を及ぼす可能性があります。 (※)「Sustainable Development Goals」の略で「持続可能な開発目標」を表すもの。   イ． 気候変動に関するリスク グローバルに取り組むべき課題の一つである気候変動については、2016年にパリ協定が発効し、日本を含む 加盟国において、温室効果ガス排出量削減のための対策が進められております。当社グループは、気候変動シ ナリオ分析を強化し、気候変動リスクへの対応策を経営レベルで検討する等、気候変動に係るリスク管理の強 化に取り組んでおります。これらの取組みは、経営会議にて決議され、取締役会及びその内部委員会であるリ スク委員会・サステナビリティ委員会においても、社外取締役等のレビューを受けながら、審議されておりま す。しかしながら、こうした取組みが奏功しない、もしくは不十分である場合、当社の財政状態及び経営成績 に悪影響を及ぼす可能性があります。 具体的には、異常気象に伴う大規模災害の発生時には、当社の従業員や店舗への直接的な被害により当社の 業務継続に支障が生じる可能性があります。また、お客さまの財務状態や担保資産の価値に悪影響を及ぼし、 結果として当社グループの与信関係費用が増加する可能性があります。 また、低炭素社会への移行に伴い、気候変動政策や規制強化、技術革新等によって気候変動に重大な影響を 与える業種の資産価値が下落する、所謂「座礁資産化」や、気候変動への対応コストの増加等に伴う貸出先の 業績悪化により、当社グループの与信関係費用が増加する可能性があります。 ― 18 ― この他、ＴＣＦＤ(※)やＳＤＧｓに係る取組みへの関心が高まるなかで、企業の社会的責任に関する取組み やその開示が不十分とみなされた場合には、当社への社会的な批判の高まりにより、当社グループの株価や資 金調達に悪影響を及ぼす可能性があります。 (※)「Task Force on Climate related Financial Disclosures」の略。2015年４月の金融安定理事会（FSB） によって設立された、気候関連財務情報開示タスクフォース。気候変動の影響を個々の企業が財務報告に おいて公表することを求めるもの。   ロ．自然資本・生物多様性に関するリスク 自然資本とは、植物や動物、大気や水や土壌などの天然資源を意味し、その毀損は、サプライチェーンを通 して、人類の生活に大きな影響を及ぼします。自然資本の安定は、生物多様性によって支えられており、その 喪失は、環境や経済、社会に広範な打撃を与えます。このような自然資本、生物多様性への関心が高まる中、 当社グループは2022年１月にＴＮＦＤ（※）フォーラムへ参画し、自然資本・生物多様性に関するリスク管 理・開示の枠組み構築に関与しながら、当社の開示の高度化に努めております。しかしながら、自然資本・生 物多様性に関する取組みや情報開示が不十分とみなされた場合には、当社への社会的な批判の高まりにより、 当社グループの株価や資金調達に悪影響を及ぼす可能性があります。 また、生物多様性を含む自然資本の喪失による自然災害の激甚化や食料不安に伴う経済・金融市場の混乱、 貸出先の業績悪化などが、当社グループの与信関係費用を増加させる可能性があります。 (※)「Task Force on Nature-related Financial Disclosures」の略。2021年６月、企業が自然資本等に関連 するリスク管理と開示枠組みを構築する為に設立された国際的な組織。企業が自然に関連した情報開示を 行うことにより、資金の流れをネイチャーポジティブ（自然へ良い影響）へ転換させることを目指してい る。   ハ．人権問題に関するリスク 当社グループは、グローバルに活動する金融機関として、人権の保護及び促進や、現代奴隷・強制労働・人 身売買・児童労働等、あらゆる形態の搾取的労働慣行に関するあらゆる活動を当社の事業及びサプライチェー ンから排除することに向けて、「人権尊重に係る声明」を公表しております。また、人権啓発推進委員会が中 心となり、全従業員の人権に対する意識を高める取組みを行っております。2022年４月には、「持続可能な調 達方針」を制定し、サプライヤーとともにサプライチェーンを含む事業活動全体における人権尊重への取組み の強化をしております。しかしながら、「ビジネスと人権に関する指導原則」(※)で明記される企業の人権尊 重責任への関心が急速に高まるなか、こうした取り組みが奏功しない、もしくは不十分である場合、お客さま 及び市場等からの信用失墜等により、当社グループの経営成績及び財政状態に影響を及ぼす可能性がありま す。 (※)2011年、国連によって採択された、人権尊重に関する国際基準。サプライチェーン上での人権侵害の発生 を防止するための「人権デューデリジェンス」等を企業に推奨。 ⑥ 他の金融機関等との競争 当社グループは、国内外の銀行、証券会社、政府系金融機関、ノンバンク等との間で熾烈な競争関係にありま す。また、今後も国内外の金融業界において金融機関同士の統合や再編、業務提携が行われる可能性や、フィン テック等の新技術の台頭により競争環境に変化が生じる可能性、他業種から金融業への進出が加速する可能性が あることに加え、金融機関に対する規制や監督の枠組みがグローバルに変更されること等により競争環境に変化 が生じる可能性があります。当社では、こうした競争環境の変化も踏まえ、2022年度までの３年間を計画期間と する中期経営計画を策定の上、様々な戦略や施策を実行してまいりますが、当社グループが競争優位を確立でき ない場合には、当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。 ― 19 ― ⑦ 各種の規制及び法制度等の変更 当社グループが国内外において業務を行う際には、様々な法律、規則、政策、実務慣行、会計制度及び税制等 の適用を受けております。当社グループではこれらの規制・法制度の動向を随時モニタリングし、適切な対応を 行っておりますが、これらが変更された場合や新たな規制等が導入された場合に、当社グループの業務運営、経 営成績及び財政状態に影響を及ぼす可能性があります。  イ.  自己資本比率規制 バーゼル銀行監督委員会は、2010年12月に、銀行の自己資本に関する国際的な基準の詳細を示す「バーゼル Ⅲ：より強靭な銀行および銀行システムのための世界的な規制の枠組み」を公表しました。 バーゼルⅢは、従来の自己資本規制(バーゼルⅡ)に比べ、優先株が普通株式等Tier１に算入されないことや Tier２に算入可能な劣後債の要件が厳格化されるなど、資本の質的側面が強化されたことに加え、自己資本比 率の最低水準の引き上げや各種バッファー(資本保全バッファー、カウンター・シクリカル・バッファー、Ｇ－ ＳＩＢバッファー)の導入により、資本の量的側面の強化を図るものであり、2013年３月期より段階的に適用さ れております。また、バーゼル銀行監督委員会は、2017年12月に、バーゼルⅢの見直しに係る最終規則文書を 公表しました。当該見直し後の規制は、当初、2022年から段階的に適用される予定でしたが、新型コロナウイ ルス感染症の感染拡大の影響を受けて、適用時期を2023年に延期する旨が、2020年３月にバーゼル銀行監督委 員会より公表されております。なお、本邦におけるバーゼルⅢの実施時期については、国際情勢や関係者との 対話を踏まえ、国際統一基準行等は、2024年３月、内部モデルを採用しない国内基準行は、2025年３月とする ことが、2022年３月に金融庁より公表されております。 当社グループは海外営業拠点を有しておりますので、連結自己資本比率を平成18年金融庁告示第20号に定め られる国際統一基準以上に維持する必要があります。また、当社の連結子会社である株式会社三井住友銀行も 海外営業拠点を有しておりますので、連結自己資本比率及び単体自己資本比率を平成18年金融庁告示第19号に 定められる国際統一基準以上に維持する必要があります。 加えて、当社の連結子会社のうち海外営業拠点を有していない株式会社ＳＭＢＣ信託銀行は、平成18年金融 庁告示第19号に定められる国内基準以上に自己資本比率を維持する必要があります。また、証券業を営むＳＭ ＢＣ日興証券株式会社は、単体ベース及びその子法人等も含めた連結ベースの自己資本比率を、金融商品取引 法等に定められている基準以上に維持する必要があります。  当社グループでは、2022年度までの３年間を計画期間とする中期経営計画の中で、バーゼルⅢの見直しに係 る最終規則文書に則った普通株式等Tier１比率(※)で10%程度を確保することを財務目標の一つとして掲げてお ります。また当社の国内銀行子会社(株式会社三井住友銀行、株式会社ＳＭＢＣ信託銀行)及びＳＭＢＣ日興証 券株式会社においても、十分な資本水準の維持に努めております。 しかしながら、当社グループ、当社の国内銀行子会社(株式会社三井住友銀行、株式会社ＳＭＢＣ信託銀行) 又はＳＭＢＣ日興証券株式会社の自己資本比率が上記の基準を下回った場合、金融庁から、自己資本の充実に 向けた様々な実行命令を自己資本比率に応じて受けるほか、業務の縮小や新規取扱いの禁止等を含む様々な命 令を受けることになります。また、海外銀行子会社については、現地において自己資本比率規制が適用されて おり、現地当局から様々な規制及び命令を受けることになります。その場合、業務が制限されること等によ り、取引先に対して十分なサービスを提供することが困難となり、その結果、当社グループの経営成績及び財 政状態に影響を及ぼす可能性があります。 （※）その他有価証券評価差額金を除く ― 20 ― ロ. ＴＬＡＣ規制他 2015年11月、金融安定理事会(ＦＳＢ)はＧ－ＳＩＢｓに対して適用される新たな規制である総損失吸収力(Ｔ ＬＡＣ)規制の枠組みを公表しました。2019年３月より、本邦における当該規制の適用が開始され、当社グルー プは、一定比率以上の総損失吸収力(ＴＬＡＣ)を維持することが求められております。 また、バーゼル銀行監督委員会は、2010年12月に、銀行の流動性に関する国際的な基準の詳細を示す「バー ゼルⅢ：流動性リスク計測、基準、モニタリングのための国際的枠組み」を公表しており、新たな規制である 流動性カバレッジ比率(ＬＣＲ)が適用されているほか、安定調達比率(ＮＳＦＲ)についても、2014年10月に最 終規則文書が公表され、2021年９月末より本邦でも導入されております。2017年12月には、バーゼルⅢの見直 しに係る最終規則文書の中で、Ｇ－ＳＩＢｓに対する追加的要件を含むレバレッジ比率規制の枠組みが最終化 されており、2019年３月から当社を含む国際統一基準行に対してレバレッジ比率の最低比率基準が導入されて おります。 こうした金融規制強化の動向を踏まえ、当社グループでは、強靭な資本基盤の構築等の施策に取り組んでお りますが、これらの施策が、企図するとおりの十分な成果を発揮しない可能性があります。 ハ. ＬＩＢＯＲ等の金利指標に関するリスク 当社グループは、お客さまの多様なニーズに的確にお応えするために各種金融サービスを提供しております が、これらの中にはロンドン銀行間取引金利（ＬＩＢＯＲ）等の金利指標を参照する金融商品が含まれており ます。また、当社グル―プは、金利リスク・為替リスクのコントロールの観点から、このような金利指標を参 照する金融商品を保有しております。 2011年以降に顕在化した、一連のＬＩＢＯＲ不正操作問題などを背景に、2017年７月には、英国の金融行動 監視機構（ＦＣＡ）長官が、2021年末以降はＬＩＢＯＲ維持のためにパネル行にレート呈示を強制する権限を 行使しない旨を表明しました。この表明を受け、日本を初めとする各国において、ＬＩＢＯＲの公表停止に向 けた取り組みが進められました。具体的には、2021年３月に、ＦＣＡ及びＬＩＢＯＲ運営機関であるＩＣＥ Ｂｅｎｃｈｍａｒｋ Ａｄｍｉｎｉｓｔｒａｔｉｏｎが公表した、現行のパネル行が呈示するレートに基づき 算出するＬＩＢＯＲの公表停止時期に従い、日本円、英ポンド、ユーロ、スイスフランの全ての公表対象期間 と米ドルの一部の公表対象期間（１週間物、２ヶ月物）については、2021年12月末をもって公表が停止済とな っており、米ドルの残りの公表対象期間については、2023年６月末をもって、公表が停止される予定です。 当社グループでは、全社的な取組みとして、適切な社内ガバナンス体制の下、お客さまへの対応や内部管理 の高度化、システム開発等の対応をグループ横断的に行っており、既に公表停止となったＬＩＢＯＲを参照と する契約を結ぶお客さまに対しては、契約更改等、対応が概ね完了しております。一方、米ドルの残りの公表 対象期間に係るＬＩＢＯＲの公表停止につきましても対応を進めておりますが、お客さまとの契約更改やシス テム開発等の対応が予定通り進まず、お客さまとの取引等に悪影響を及ぼす可能性や追加のシステム開発費用 が発生する可能性があります。また、参照金利の変更、金融商品の評価方法の変更等により、当社グループの 保有する金利指標を参照する金融商品に損失が発生する可能性があります。これらの結果、当社グループの経 営成績及び財政状態に影響を及ぼす可能性があります。 ― 21 ― (2) 当社グループの業務に内包されるリスク 当社グループは、銀行業務を中心に、リース業務、証券業務、コンシューマーファイナンス業務等の各種金融サ ービスを行うグループ会社群によって構成されており、これらの会社で相互に協働して営業活動を行っております が、業務遂行にあたり以下のようなリスクを認識しております。 ① 信用リスク 信用リスクとは、与信先の財務状況の悪化等のクレジットイベント（信用事由）に起因して、資産（オフバラ ンス資産を含む）の価値が減少又は滅失し、損失を被るリスクであります。当社グループでは、後記「第５ 経理 の状況 １ 連結財務諸表等 注記事項 (金融商品関係) １ 金融商品の状況に関する事項 (3) 金融商品に 係るリスク管理体制 ① 信用リスクの管理」に記載のとおり、適切なリスク管理体制を構築しておりますが、 取引先の業況の悪化やカントリーリスクの高まり等に伴い、幅広い業種で貸倒引当金及び貸倒償却等の与信関係 費用や不良債権残高が増加し、当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。 イ. 取引先の業況の悪化 当社グループの取引先の中には、当該企業の属する業界が抱える固有の事情等の影響を受けている企業があ りますが、国内外の経済金融環境及び特定業種の抱える固有の事情の変化等により、当該業種に属する企業の 財政状態が悪化する可能性があります。また、当社グループは、債権の回収を極大化するために、当社グルー プの貸出先に対する債権者としての法的権利を必ずしも行使せずに、状況に応じて債権放棄、デット・エクイ ティ・スワップ又は第三者割当増資の引受、追加貸出等の金融支援を行うことがあります。これら貸出先の信 用状態が悪化する、又は企業再建が奏功しない場合には、当社グループの与信関係費用や不良債権残高が増加 する可能性があります。 ロ. 他の金融機関における状況の変化 世界的な市場の混乱等により、国内外の金融機関の経営状態が悪化し、資金調達及び支払能力等に問題が生 じた場合には、当社グループが問題の生じた金融機関への支援を要請される可能性がありますが、当該金融機 関の信用状態に改善が見られない場合には、当社グループの与信関係費用や不良債権残高が増加する可能性が あります。また、他の金融機関による貸出先への融資の打ち切りや回収があった場合にも、当該貸出先の経営 状態の悪化により、当社グループの与信関係費用や不良債権残高が増加する可能性があり、それらの結果、当 社グループの経営成績及び財政状態に影響を及ぼす可能性があります。 ② 市場リスク 市場リスクとは、金利・為替・株式等の相場が変動することにより、金融商品の時価が変動し、損失を被るリ スクであります。当社グループでは、後記「第５ 経理の状況 １ 連結財務諸表等 注記事項 (金融商品関係) １ 金融商品の状況に関する事項 (3) 金融商品に係るリスク管理体制 ② 市場リスク・流動性リスクの管 理」に記載のとおり、適切なリスク管理体制を構築しておりますが、急激な相場の変動等により、保有する金融 資産で多額の評価損・減損等が発生し、結果として当社グループの経営成績及び財政状態に影響を及ぼす可能性 があります。 ― 22 ― イ. 金利変動リスク 当社グループは、国債等の市場性のある債券やデリバティブ等の金融商品を保有しております。これらは金 利変動によりその価格が変動するため、主要国の金融政策の変更や、債券等の格付の低下、世界的な市場の混 乱や金融経済環境の悪化等により金利が変動した場合、多額の売却損や評価損等が発生し、当社グループの経 営成績及び財政状態に影響を及ぼす可能性があります。 ロ. 為替変動リスク 当社グループは、保有する外貨建資産及び負債について、必要に応じて、為替リスクを回避する目的からヘ ッジ取引を行っておりますが、為替レートが急激に大きく変動した場合等には、多額の為替差損等が発生し、 当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。 ハ. 株価変動リスク 当社グループは、市場性のある株式等、大量の株式を保有しております。国内外の経済情勢や株式市場の需 給関係の悪化、発行体の経営状態の悪化等により株価が低下する場合には、保有株式に減損又は評価損が発生 し、当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。また、当社グループは、大幅な 株価下落をもたらすストレス環境下においても十分に金融仲介機能を発揮できる財務基盤を確保する観点か ら、政策保有株式の削減計画を策定し、本計画に取り組んでおります。この株式削減に伴い、売却損失が発生 する可能性があるほか、取引先が保有する当社株式が売却されることで、当社の株価に悪影響を及ぼす可能性 があります。 ③ 流動性リスク 流動性リスクとは、運用と調達の期間のミスマッチや予期せぬ資金の流出により、決済に必要な資金調達に支 障をきたす、もしくは通常より著しく高い金利での調達を余儀なくされるリスクです。 当社グループでは、後記 「第５ 経理の状況 １ 連結財務諸表等 注記事項 (金融商品関係) １ 金融商品の状況に関する事項 (3) 金融商品に係るリスク管理体制 ② 市場リスク・流動性リスクの管理」に記載のとおり、適切なリスク管理体 制を構築しておりますが、当社グループ各社の格付が低下した場合には、当社グループの国内外における資本及 び資金調達の条件が悪化する、もしくは取引が制約される可能性があります。また、世界的な市場の混乱や金融 経済環境の悪化等の外部要因によっても、当社グループの国内外における資本及び資金調達の条件が悪化する、 もしくは取引が制約される可能性があります。このような事態が生じた場合、当社グループの資本及び資金調達 費用が増加したり、外貨資金調達等に困難が生じたりする等、当社グループの経営成績及び財政状態に影響を及 ぼす可能性があります。 ― 23 ― ④ オペレーショナルリスク オペレーショナルリスクとは、内部プロセス・人・システムが不適切であること、もしくは機能しないこと、 又は外生的事象が生起することから生じる損失にかかるリスクであり、具体的には、以下のとおりであります。 イ. 事務リスク 当社グループは、事務に関する社内規程等の整備、事務処理のシステム化、本部による事務指導及び事務処 理状況の点検等により適正な事務の遂行に努めておりますが、役職員等が事務に関する社内規程等に定められ たとおりの事務処理を怠る、あるいは事故・不正等を起こした場合には、当社グループの経営成績及び財政状 態に影響を及ぼす可能性があります。 ロ. 情報システム・サイバー攻撃に関するリスク 当社グループが業務上使用している情報システムにおいては、安定的な稼働を維持するためのメンテナン ス、バックアップシステムの確保等の障害発生の防止策を講じ、また、不測の事態に備えたコンティンジェン シープランを策定し、システムダウンや誤作動等の障害が万一発生した場合であっても安全かつ速やかに業務 を継続できるよう体制の整備に万全を期しております。しかしながら、これらの施策にもかかわらず、品質不 良、人為的ミス、サイバー攻撃等外部からの不正アクセス、コンピューターウィルス、災害や停電、テロ等の 要因によって、情報システムに、システムダウン、誤作動、不備、不正利用を含む障害が発生する可能性があ ります。 特に、近年のデジタル技術の著しい発展により、インターネットやスマートフォンを利用した取引が増加し ている一方、サイバー攻撃手法の高度化・巧妙化も急速に進展しており、金融機関をとりまくサイバーリスク はより一層深刻化しております。加えて、取引先や業務委託先等の第三者のシステムを経由したサイバーリス クにも直面しております。 以上の認識の下、当社グループは、経営主導でサイバー攻撃に対するセキュリティ対策の強化をより一層推 進することを定めた「サイバーセキュリティ経営宣言」を策定しており、経営会議・取締役会での議論・検証 の下、適切なリソースを配分するほか、サイバーセキュリティ専担組織を設置し、外部機関と連携した脅威情 報の収集、24時間365日監視体制の構築、サイバー攻撃に対する多層防御やウイルス侵入も想定したセキュリテ ィ対策の導入等、継続的なレベルアップ施策を講じてきておりますが、これらの方策も最新の攻撃に対しては 万全でない可能性があります。 これらの要因により、当社の情報システムに障害が発生した場合、当社グループの経営成績及び財政状態に 影響を及ぼす可能性があります。 ハ. お客さまに関する情報の漏洩 当社グループは、情報管理に関する規程及び体制の整備や役職員に対する教育の徹底等により、お客さまに 関する情報の管理には万全を期しております。また、業務委託先である外部業者が、お客さまに関する情報を 取り扱う場合には、外部業者の情報管理体制やシステムセキュリティ管理体制を検証し、情報管理が適切にな されていることを確認しております。しかしながら、内部又はサイバー攻撃等外部からのコンピューターへの 不正アクセスや、役職員や外部業者等の人為的ミス、事故、不正等が原因で、お客さまに関する情報が外部に 漏洩した場合、お客さまからの損害賠償請求やお客さま及び市場等からの信頼失墜等により、当社グループの 経営成績及び財政状態に影響を及ぼす可能性があります。 ― 24 ― ニ. 重要な訴訟等 当社グループは、国内外において、銀行業務を中心に、リース業務、証券業務、コンシューマーファイナン ス業務等の各種金融サービスを行うグループ会社群によって構成されており、付加価値の高い金融サービスを 幅広く提供しております。こうした業務遂行の過程で、損害賠償請求訴訟等を提起されたり、損害に対する補 償が必要となる可能性があります。当社グループでは、訴訟が提起された場合等においては、弁護士の助言等 に基づき、事態の調査を行い、適切な対応方針を策定の上、代理人を選任し、適切に訴訟手続を遂行しており ます。また、経営に重大な影響を与えると認められる訴訟等については、監査委員会、取締役会及びグループ 経営会議に報告しております。しかしながら、これらの取組みにも関わらず、訴訟等の結果によっては、当社 グループの経営成績及び財政状態に影響を及ぼす可能性があります。  ⑤ コンダクトリスク コンダクトリスクとは、法令や社会規範に反する行為等により、顧客保護・市場の健全性・公正な競争・公共 の利益及び当社グループのステークホルダーに悪影響を及ぼすリスクを指します。当社グループは、経営上の重 大なリスクを特定・評価し、コントロール策によるリスクの低減・制御を図っています。また、役職員に対する 研修等を通じ、健全なリスクカルチャーの浸透・醸成に努めています。しかしながら、これらの取組みにも関わ らず、役職員等の不適切な行為が原因で、市場及び公共の利益等に悪影響を与えた場合、お客さま及び市場等か らの信用失墜等により、当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。なお、当該リ スクの内、法令等に違反するリスク、経済制裁対象国との取引に係るリスクについては以下の通りとなります。 イ. 法令等に違反するリスク 当社グループは業務を行うにあたり、会社法、銀行法、独占禁止法、金融商品取引法、貸金業法、外為法、 犯罪収益移転防止法及び金融商品取引所が定める関係規則等の各種法規制の適用を受けております。また、海 外においては、それぞれの国や地域の規制・法制度の適用、及び金融当局の監督を受けております。加えて、 各国当局は、マネー・ローンダリング及びテロ資金供与防止に関連し、ＦＡＴＦ等の国際機関の要請に基づい た各種施策を強化しており、当社グループは、国内外で業務を行うにあたり、これらの各国規制当局による各 種規制の適用を受けております。さらに、当社は、米国証券取引所上場会社として、米国サーベンス・オクス リー法や米国証券法、米国海外腐敗行為防止法等の各種法制の適用を受けております。 当社グループは、法令その他諸規則等を遵守すべく、コンプライアンス体制及び内部管理体制の強化を経営 上の最重要課題のひとつとして位置付け、グループ各社の役職員等に対して適切な指示、指導及びモニタリン グを行う体制を整備するとともに、不正行為の防止・発見のために予防策を講じております。しかしながら、 当社グループにおいて、法令その他諸規則等を遵守できなかった場合、法的な検討が不十分であった場合又は 予防策が効果を発揮せず役職員等による不正行為が行われた場合には、不測の損失が発生したり、行政処分や 罰則を受けたり、業務に制限を付されたりするおそれがあり、また、お客さまからの損害賠償請求やお客さま 及び市場等からの信頼失墜等により、当社グループの経営成績及び財政状態に影響を及ぼす可能性がありま す。 なお、当社の連結子会社であるＳＭＢＣ日興証券株式会社において、2022年３月24日及び４月13日、東京地 方検察庁により、同社元執行役員及び同社社員並びに法人としての同社が、違法な安定操作取引の疑いにて、 それぞれ東京地方裁判所へ起訴されました。 今後、同社には金融商品取引法に規定する罰金刑等が発生する可能性や、行政処分の発令等により、同社の 業務に影響が生じる可能性がある他、同社を含めた当社グループに対するお客さま及び市場等からの信頼失墜 により、ビジネス機会の喪失等が発生する可能性があります。これらにより当社グループの経営成績及び財政 状態に悪影響を及ぼす可能性があります。 ― 25 ― ロ. 経済制裁対象国との取引に係るリスク 本邦を含む各国当局は、経済制裁対象国や特定の団体・個人等との取引を制限しております。例えば、米国 関連法規制の下では、米国政府が経済制裁対象国と指定している国等と米国人(米国内の企業を含む)が事業を 行うことを、一般的に禁止又は制限しております。また、米国政府は、イラン制裁関連法制等により、米国以 外の法人、個人に対しても、イランの指定団体や指定金融機関との取引等を規制しております。当社グループ は、本邦・米国を含む各国の法規制を遵守する体制を整備しておりますが、既に米国財務省外国資産管理室(Ｏ ＦＡＣ)に自主開示している取引を含めて、当社グループが行った事業が法規制に抵触した場合には、関連当局 より過料等の処分を受ける可能性や厳しい行政処分等を受ける可能性があります。なお、取引規模は限定的で ありますが、当社の銀行子会社の米国以外の拠点において、米国法令等を含む各国関連法規の遵守を前提とし て、経済制裁対象国と銀行間取引を行う場合があり、経済制裁対象国との取引が存在すること等により当社グ ループの風評が悪化し、お客さまや投資者の獲得あるいは維持に支障を来す可能性があります。それらによ り、当社グループの株価、業務、経営成績及び財政状態に影響を及ぼす可能性があります。 ⑥ 決済リスク 当社グループは、国内外の多くの金融機関と多様な取引を行っております。大規模なシステム障害や災害が発 生した場合、政治的な混乱等により取引相手である金融機関の決済が行われないような事態等が発生した場合、 又は金融システム不安が発生した場合に、金融市場における流動性が低下する等、決済が困難になるリスクがあ ります。また、非金融機関の取引先との一定の決済業務においても取引先の財政状態の悪化等により決済が困難 になるリスクがあります。 当社グループでは、勘定系システム等の重要なシステムについては、バックアップサーバーを東日本・西日本 に分散して設置するとともに、定期的な訓練を実施する等、システム障害や災害発生時に迅速に対応できる体制 の構築に努めているほか、日中の流動性について、定期的なモニタリングやストレステストの実施等、当社グル ープの決済が滞らないよう管理する体制を構築しております。 しかしながら、想定を上回る事態が発生した場合には、決済が困難になることで、当社グループの経営成績及 び財政状態に影響を及ぼす可能性があります。 ⑦レピュテーショナルリスク 当社グループでは、レピュテーショナルリスクが顕在化するおそれがある事態に関する情報を適切に収集する と共に、このような事態に対して適切な措置を講ずることにより、リスクの制御及び削減に努めております。し かしながら、これらの取組みにも関わらず、当社グループの事業や従業員その他関係者の行為により、お客さま や株主をはじめとするステークホルダーからの高い期待に応えられず、当社グループの企業価値の毀損や信頼低 下に繋がる可能性があります。 ⑧モデルリスク モデルリスクとは、モデル（※）の開発若しくは実装での作業ミス、または、モデルの前提や限界を超えた利 用等により、経営判断・業務判断等を誤り、損失・不利益を被るリスクを指します。当社グループでは、リスク 管理や時価評価等にモデルを活用しており、モデルの開発・使用等の各プロセスに応じた適切な管理を実施する ことで、モデルリスクの低減を図っておりますが、モデル開発時の想定を超えた金融経済環境、事業環境の変化 に直面したり、役職員による不適切なモデル利用がなされた場合等は、モデルのアウトプットの不確実性が高ま り、経営判断・業務判断を誤る可能性があります。 (※)統計学的、経済学的、財務的、数理的な理論・手法・過程を用いて、入力データを処理し、推定値・予測 値・分類等を出力する定量的手法。 ― 26 ― ⑨ 戦略リスク イ. 当社グループのビジネス戦略に関するリスク 当社グループは、銀行業務を中心に、リース業務、証券業務、コンシューマーファイナンス業務等の各種金 融サービスを行うグループ会社群によって構成されており、中長期ビジョンとして、「最高の信頼を通じて、 お客さま・社会とともに発展するグローバルソリューションプロバイダー」を掲げ、2020年５月に公表した、 2020年度から2022年度までの３年間を計画期間とする中期経営計画において、このビジョンの実現に向けた 様々なビジネス戦略を実施してまいります。これらのビジネス戦略は、後記「(3)トップリスク」に記載の、経 営上特に重要なリスク事象も踏まえ策定しておりますが、想定外の金融経済環境、事業環境の変化等により、 必ずしも奏功するとは限らず、当初想定した成果をもたらさない可能性があります。 ロ. 当社の出資、戦略的提携等に係るリスク 当社グループはこれまで、銀行業務、リース業務、証券業務、コンシューマーファイナンス業務等における 様々な戦略的提携、提携を視野に入れた出資、買収等を国内外で行ってきており、今後も同様の戦略的提携等 を行っていく可能性があります。当社グループでは、これらの戦略的提携等を行うにあたっては、そのリスク や妥当性を十分に検討しておりますが、①法制度の変更、②金融経済環境の変化や競争の激化、③提携先や出 資・買収先の業務遂行に支障をきたす事態が生じた場合等には、期待されるサービス提供や十分な収益を確保 できない可能性があります。また、当社グループの提携先又は当社グループのいずれかが、戦略を変更し、相 手方との提携により想定した成果が得られないと判断し、あるいは財務上・業務上の困難に直面すること等に よって、提携関係が解消される場合には、当社グループの収益力が低下したり、提携に際して取得した株式や 提携により生じたのれん等の無形固定資産、提携先に対する貸出金の価値が毀損したりする可能性がありま す。これらの結果、当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。 ハ. 戦略遂行に必要な有能な人材の確保 当社グループは幅広い分野で高い専門性を必要とする業務を行っておりますので、各分野において有能で熟 練した人材が必要とされます。当社グループでは、役職員の積極的な採用及び役職員の継続的な研修等によ り、多様な人材の確保・育成を行っておりますが、有能な人材を継続的に採用し定着を図ることができなかっ た場合には、戦略・主要分野での人材確保が困難となり、策定したビジネス戦略が想定通りに実施できない可 能性があります。その結果、当社グループの経営成績及び財政状態に影響を及ぼす可能性があります。 ⑩ 財務報告に係る内部統制に関するリスク 当社は、金融商品取引法に基づいて、財務報告に係る内部統制の有効性を評価し、その結果を記載した内部統 制報告書の提出を義務付けられております。また、当社は、米国証券取引所上場会社として、米国サーベンス・ オクスリー法に基づいて、財務報告に係る内部統制等の評価も義務付けられております。 当社は、会計処理の適正性及び財務報告の信頼性を確保するため、財務報告に係る内部統制評価規程等を制定 し、財務報告に係る内部統制について必要な体制を整備しております。しかしながら、財務報告に係る内部統制 が有効でない場合には、当社の財務報告に対するお客さま及び投資者等からの信頼を損ない、その結果、当社の 株価が悪影響を受ける可能性があります。 ⑪ リスク管理方針及び手続の有効性に関するリスク 当社グループは、リスク管理方針及び手続を整備し運用しておりますが、新しい分野への急速な業務の進出や 拡大に伴い、リスク管理方針及び手続が有効に機能しない可能性があります。また、当社グループのリスク管理 方針及び手続の一部は、過去の経験に基づいた部分があることから、将来発生する多様なリスクを必ずしも正確 に予測することができず、有効に機能しない可能性があります。その結果、当社グループの経営成績及び財政状態に影"

In [65]:
nlp = spacy.load("ja_ginza")

#名詞、動詞、形容詞のみ抽出
include_pos = ('NOUN', 'VERB', 'ADJ')
#不要語を指定
stopwords = ('する', 'ある', 'ない', 'いう', 'もの', 'こと', 'よう', 'なる', 'ほう', 'いる', 'くる')


def extract_words(sent, pos_tags, stopwords):
    words = [token.lemma_ for token in sent
             if token.pos_ in pos_tags and token.lemma_ not in stopwords]
    return words

doc = nlp(text)
sents = [extract_words(sent, include_pos, stopwords)
          for sent in doc.sents]

## Word2Vecの学習
* ハイパーパラメータ  
https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec

In [79]:
model = Word2Vec(sentences=sents, vector_size=100, window=5, min_count=1, workers=4)

## モデルの保存

In [80]:
model.save("word2vec.model")

## モデルの読み込み

In [81]:
model = Word2Vec.load("word2vec.model")

## 単語の分散表現の出力

In [75]:
model.wv['人権']

array([-0.00639947, -0.0013484 ,  0.00823085,  0.01261581,  0.01034695,
       -0.01774785,  0.00788717,  0.01720633,  0.000269  , -0.01266967,
       -0.00088427, -0.01317626, -0.00893227,  0.00485134,  0.01072847,
        0.00491929, -0.00604498, -0.00470532, -0.00570053, -0.0023647 ,
       -0.0113332 ,  0.01167607, -0.0057797 , -0.01216068, -0.00100039,
        0.0018794 , -0.01220319, -0.00379782, -0.00613234, -0.0043695 ,
        0.01142897,  0.00382953,  0.01083189,  0.00576303, -0.01154621,
        0.01539006,  0.00893597,  0.00296345,  0.00296708, -0.01356735,
        0.00404362, -0.00675977, -0.00309513, -0.00626204,  0.00536447,
        0.00511377, -0.01144777,  0.00659066,  0.00368828,  0.00474846,
        0.00089533, -0.00469918,  0.0082931 ,  0.00651455, -0.01075827,
        0.01183242,  0.00019744, -0.00878655,  0.00399527,  0.00686002,
        0.00332193,  0.01130093, -0.00441325, -0.00773927, -0.00833736,
        0.00085107,  0.00325703,  0.00825082, -0.01462528,  0.01

## 類義語の探索

In [77]:
results = model.wv.most_similar(positive="人権", topn=10)
for result in results:
    print(result[0], 't', result[1])

業務 t 0.7057178020477295
等 t 0.701554000377655
法 t 0.6976000666618347
成績 t 0.6894018054008484
当社 t 0.6843270659446716
先 t 0.677718997001648
発生 t 0.6756765842437744
財務 t 0.6748518943786621
整備 t 0.6690458059310913
第 t 0.6674997806549072


## モデルからkeyとベクトルの情報だけ分離する
* 追加の学習が不要な場合はベクトルとキーだけを保持することで高速に処理することができる

In [82]:
# wordとembeddingの情報を保存
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [83]:
vector = wv['人権'] 
vector

array([-1.49975909e-04,  8.81097279e-03, -5.90819912e-03,  6.93876669e-03,
        1.04771033e-02, -6.55906915e-04,  4.45921160e-03,  1.02666551e-02,
       -3.72007210e-03, -9.18891188e-03, -1.33471694e-05, -2.75655044e-03,
       -7.21443351e-03,  2.98660877e-03,  5.42030064e-03, -8.67962651e-03,
        2.11071991e-03,  5.28045371e-03, -9.64045618e-03, -7.92567385e-04,
       -7.72961741e-03, -6.92486297e-03,  7.98644032e-03, -7.73001695e-03,
       -2.66969926e-03, -7.41314515e-03, -1.07429251e-02, -1.00970790e-02,
       -1.87021180e-03,  1.95623911e-03, -4.99333488e-03, -8.05220846e-03,
       -1.21638842e-03, -3.99693940e-03, -8.22363514e-03, -8.06514407e-04,
        3.49358749e-03,  8.56456056e-04,  4.66276100e-03, -1.22696022e-03,
       -6.53003051e-04,  8.41356814e-04,  7.63184624e-03,  1.21278479e-03,
        4.31419211e-03, -1.49061932e-04, -1.04714269e-02,  4.98470617e-03,
        6.96426164e-03,  1.90007954e-03,  1.08719645e-02, -7.91685563e-03,
        3.02903191e-03, -